In [ ]:
import pandas as pd
import numpy as np
import math
import random
from tqdm.auto import tqdm
import torch

# Module

In [ ]:
import numpy as np

class rbfSVMClassifier:
    def __init__(self,n_iters=5, lr = 0.0001, random_seed=3, sigma=1):
        self.n_iters = n_iters # 몇 회 반복하여 적절한 값을 찾을지 정하는 파라미터
        self.lr = lr  # 학습률과 관련된 파라미터 
        self.sigma = sigma
        self.random_seed = random_seed
        np.random.seed(self.random_seed)

    def rbf_kernel(self,x,z):
        numer = -(x-z).dot(x-z)
        denom = 2*self.sigma**2
        return math.exp(numer/denom) 
    
    def sign(self,scalar):
        if scalar >= 0:
            return 1
        else:
            return 0
    
    def update_weight(self,x_i,y_i,i):
        summ = 0
        if i == 0:
            b_next = self.lr*(y_i-self.sign(0))
            self.weight = np.append(self.weight,b_next)
            return self.weight
        else:
            for j in range(i):
                summ += self.weight[j]*self.rbf_kernel(self.train_x[j],x_i)
            b_next = self.lr*(y_i-self.sign(summ))
            self.weight =np.append(self.weight,b_next)
            return self.weight
    
    def fit(self, x_, y_orig):
        """
        본 함수는 x, y를 활용하여 훈련하는 과정을 코딩하는 부분입니다.
        아래 reference 사이트의 gradient 계산 부분을 참고하세요.
        reference: https://towardsdatascience.com/support-vector-machine-introduction-to-machine-learning-algorithms-934a444fca47
        아래 총 6개의 None 부분을 채우시면 됩니다.

        """
        n_samples, n_features = x_.shape
        # hint: y값을 SVM 계산에 활용해주기 위하여 0에 해당하는 y값들을 -1로 변환
        y_ =  np.array(y_orig)#numpy array의 y를 y_에 담아줍니다       
        y_[y_ == 0] = -1#0에 해당하는 y_값들을 -1로 변환합니다.
        
        self.weight = np.array([])
        
        n_label = np.sum((y_==1)*1)
        
        for _ in tqdm(range(self.n_iters),leave =False, colour = "cyan", desc = f"training for {self.n_iters} iterations"):
            
            ###### Oversampling #######
            non_label_indices = np.where(y_ == -1)[0]
            label_indices = np.where(y_ != -1)[0]
            more_sample_n = len(non_label_indices)-len(label_indices)
            n_iter = _
            if more_sample_n <0:
                more_sample_indices = np.random.choice(non_label_indices,size = -more_sample_n,replace = False)
            else:
                more_sample_indices = np.random.choice(label_indices,size = more_sample_n,replace = True)
            x = np.vstack([x_,x_[more_sample_indices,:]])
            y = np.concatenate([y_,y_[more_sample_indices]])
            self.train_x = x
            self.train_y = y 
            n_samples_oversampled = len(y)
            assert Counter(y)[list(Counter(y).keys())[0]] == Counter(y)[list(Counter(y).keys())[1]]
            assert x.shape[0] == y.shape[0]
            for i in tqdm(range(n_samples_oversampled),leave = False,colour = "green",desc = "updating through all train data"):
                x_i = x[i]
                y_i = y[i]
                #### Update considering gradient
                self.weight = self.update_weight(x_i,y_i,i)
                
        return self.weight
    
    def predict_(self,x):#predict for single instance
        summ = 0
        for idx, x_ in enumerate(self.train_x):
            summ += self.weight[idx]*self.rbf_kernel(x,x_)
        return self.sign(summ)
    
    def predict(self, x):
        n_test_data = x.shape[0]
        preds = []
        for i in range(n_test_data):
            pred = self.predict_(x[i,:])
            preds.append(pred)
        preds = np.array(preds)
        return preds#예측 값을 담은 배열인 approx를 반환합니다
    
    def sigmoid(self,z):
        return 1./(1.+np.exp(-z))
    
    def get_prob_(self,x_i):
        summ = 0
        for idx, x_ in enumerte(self.train_x):
            summ += self.weight[idx]*self.rbf_kernel(x_i,x_)
        return self.sigmoid(summ)
    
    def get_prob(self,X):
        n_test_data = x.shape[0]
        probs = []
        for i in range(n_test_data):
            prob = self.get_prob_(x[i,:])
            probs.append(prob)
        probs = np.array(probs)
        return probs


    def get_accuracy(self, y_true, y_pred):
        """
            y_true, y_pred가 들어왔을 때, 정확도를 계산하는 함수.
            sklearn의 accuracy_score 사용 불가능 / sklearn의 accuracy_score 함수를 구현한다고 생각하면 됩니다.
            넘파이만을 활용하여 정확도 계산 함수를 작성하세요.
        """
        acc = np.sum(y_true == y_pred)/len(y_pred)#예측하고자 하는 전체 데이터 수 개수(len(y_pred)) 중 예측 값과 실제 값이 일치하는 데이터 개수(np.sum(y_true == y_pred))의 비율로 정확도를 계산합니다
        return acc #정확도를 담은 acc를 반환합니다


class MultiClassSVM:
    def __init__(self, kernel='rbf', C=1, gamma=0.1, n_iters = 3):
        self.kernel = kernel
        self.C = C
        self.gamma = gamma
        self.svms = []
        self.n_iters = n_iters
        self.n_classes = 4

    def fit(self, X, y):
        self.svms = []
        n_classes = self.n_classes
        for i in tqdm(range(n_classes),leave = False, colour="pink",desc = "exploring all classes.."):
            y_temp = np.where(y == i, 1, -1)
            svm = rbfSVMClassifier(lr=self.C, sigma=self.gamma, n_iters = self.n_iters)
            svm.fit(X, y_temp)
            self.svms.append(svm)
        assert len(self.svms) == self.n_classes
    def predict(self, X):
        predictions = np.zeros((X.shape[0], self.n_classes))
        for i, svm in enumerate(self.svms):
            predictions[:, i] = svm.predict(X)
        return np.argmax(predictions, axis=1)

In [ ]:
train_df = pd.read_csv("../data/train.csv")
train_df.head()

,traffic(t-10),traffic(t-9),traffic(t-8),traffic(t-7),traffic(t-6),traffic(t-5),traffic(t-4),traffic(t-3),traffic(t-2),traffic(t-1),traffic(t),type
0,3.0,2.0,5.0,6.0,6.0,4.0,6.0,6.0,3.0,12.0,6.0,0
1,2.0,5.0,6.0,6.0,4.0,6.0,6.0,3.0,12.0,6.0,6.0,0
2,5.0,6.0,6.0,4.0,6.0,6.0,3.0,12.0,6.0,6.0,4.0,0
3,6.0,6.0,4.0,6.0,6.0,3.0,12.0,6.0,6.0,4.0,6.0,0
4,6.0,4.0,6.0,6.0,3.0,12.0,6.0,6.0,4.0,6.0,17.0,0


In [ ]:
X,y = train_df.drop('type',axis = 1).to_numpy(),train_df['type']

In [ ]:
X_scaled = (X-X.mean())/X.std()

In [ ]:
test_df = pd.read_csv("../data/test.csv")
test_df.head()

,traffic(t-10),traffic(t-9),traffic(t-8),traffic(t-7),traffic(t-6),traffic(t-5),traffic(t-4),traffic(t-3),traffic(t-2),traffic(t-1),traffic(t)
0,61.0,26.0,29.0,37.0,20.0,32.0,25.0,25.0,16.0,18.0,22
1,2.0,10.0,11.0,28.0,21.0,18.0,5.0,6.0,10.0,3.0,7
2,14.0,1.0,14.0,14.0,14.0,7.0,20.0,14.0,9.0,7.0,20
3,20.0,7.0,25.0,15.0,21.0,13.0,21.0,14.0,8.0,20.0,14
4,155.0,71.0,10.0,23.0,480.0,617.0,58.0,1019.0,166.0,2.0,2


In [ ]:
X_test = test_df.to_numpy()
X_test_scaled = (X_test-X.mean())/X.std()

In [ ]:
y.shape

(12645,)

In [ ]:
model_ckpt = torch.load("./rbf_multiclass0.90.pt")
model_ckpt.keys()

dict_keys(['C', 'gamma', 'train_acc', 'test_acc', 'best_model'])

In [ ]:
model = model_ckpt['best_model']

In [ ]:
model.n_iters = 20

In [ ]:
from collections import Counter
model.fit(X_scaled,y)

exploring all classes..:   0%|          | 0/4 [00:00<?, ?it/s]

training for 20 iterations:   0%|          | 0/20 [00:00<?, ?it/s]

updating through all train data:   0%|          | 0/15772 [00:00<?, ?it/s]

updating through all train data:   0%|          | 0/15772 [00:00<?, ?it/s]

updating through all train data:   0%|          | 0/15772 [00:00<?, ?it/s]

...

updating through all train data:   0%|          | 0/23474 [00:00<?, ?it/s]

updating through all train data:   0%|          | 0/23474 [00:00<?, ?it/s]

updating through all train data:   0%|          | 0/23474 [00:00<?, ?it/s]

updating through all train data:   0%|          | 0/23474 [00:00<?, ?it/s]

updating through all train data:   0%|          | 0/23474 [00:00<?, ?it/s]

In [ ]:
train_preds = model.predict(X_scaled)

In [ ]:
np.mean(train_preds == y)#accuracy updated!

0.9193357058125742

In [ ]:
Counter(y[train_preds != y])

Counter({1: 203, 2: 207, 3: 610})

In [ ]:
np.unique(train_preds)

In [ ]:
test_preds = model.predict(X_test_scaled)

In [ ]:
np.unique(test_preds)

In [ ]:
from collections import Counter
Counter(test_preds)

In [ ]:
np.mean(test_preds == y_pred)

In [ ]:
sample_csv = pd.read_csv("../data/sample_submission.csv")
sample_csv['type'] = test_preds

In [ ]:
sample_csv.to_csv("../submissions/ch_submit.csv",index = False)